

# Project: The Attendance of the Appointments 

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

## Introduction 
This dataset is about 110527 appointments in Brazil with 14 features, and is focused on the question of whether or not patients attende for their appointment.
- "Gender" 'F' for Female and 'M' for Male.
- "ScheduledDay" on what day the patient set up their appointment.
- "AppointmentDay" on what day the patient book the appointment.
- "Neighborhood" the location of the hospital.
- "Scholarship" did the patient enrolled in scholarship or not ?



<a id='intro'></a>

 


## Question for Analysis
#### Is the SMS reminder for the appiontment is affect on the attendance ? 
#### Is the Difference between scheduled day and appointment day affect on Attendance ? 

We will answer the questions below..

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
% matplotlib inline

<a id='wrangling'></a>
## Data Wrangling

In [ ]:
df = pd.read_csv('no_show_appointments.csv')
df.head()

In [ ]:
df.shape #the number of rows and columns

the number of appointments is 110527 and the number of features is 14

In [ ]:
df.info() 

In [ ]:
df.isnull().sum() #check for missing values

There is no any missing value

In [ ]:
df.duplicated().sum()

there is no duplicated rows here!

In [ ]:
df ['AppointmentID'].duplicated().sum() #the number of duplicated appointment id 

In [ ]:
df['PatientId'].duplicated().sum() #the number of patients that have been came more than once

There is 48783 patients who's came to the appointment more than once

In [ ]:
df.describe()

- The most of patients they didn't have any disease 
- The age mean is 37 
- Only 32% of patients received a reminding messege.
- The minimum age is -1, so we should delete the row.

In [ ]:
mask = df.query('Age==-1') #the row of -1 age
mask

In [ ]:
df.dtypes #the types of data

### Data cleaning 

In [ ]:
df.drop(index=99832 ,inplace=True)

In [ ]:
df.drop_duplicates('PatientId' , inplace=True) #to drop the rows of patients that have been came more than once 
df.shape

In [ ]:
df.drop(['PatientId', 'AppointmentID' , 'Neighbourhood'], axis = 1, inplace = True) #drop the columns that we don't need
df.head()

In [ ]:
df['ScheduledDay'] = pd.to_datetime(df['ScheduledDay']) #change the type of scheduled day column from object to datetime
df['AppointmentDay'] = pd.to_datetime(df['AppointmentDay']) #change the type of appointment day column from object to datetime
df.dtypes

In [ ]:
df['ScheduledDay'] = pd.to_datetime(df['ScheduledDay']).dt.normalize() #remove the time from scheduled day column
df['AppointmentDay'] = pd.to_datetime(df['AppointmentDay']).dt.normalize() #remove the time from appointment day column
df.head()

In [ ]:
df.rename(columns = {'No-show':'Attendance'}, inplace = True) #change the column name from no-show to attendance just to be clear

In [ ]:
df = df.copy()
df['Attendance'] = df['Attendance'].map({'Yes': False, 'No': True}) #convert attendance column to boolean 

In [ ]:
df.info()

Attendance column is clear now.

#### Awesome! The data is clean. 
We drop the duplicated patient id and the columns that we don't need, then we change some types of data, and rename the dependent column.

<a id='eda'></a>
## Exploratory Data Analysis



In [ ]:
df.hist(figsize=(15,8));



### Is the SMS reminder for the appiontment is affect on the attendance ?

In [ ]:
df['Attendance'].value_counts() #the number of attende and absente patients

In [ ]:
df.groupby(['Attendance'])['SMS_received'].sum() #the number of attende and absente patients who's received a reminding message

only 14760 out of 49653 of atteneded patients they received a reminding message.

In [ ]:
df.groupby(['Attendance'])['SMS_received'].mean() #the means of attende and absente patients who's received a reminding message

In [ ]:
df.groupby('Attendance')['SMS_received'].mean().plot( kind = 'bar' );
plt.title('The effect of a reminding message on Attendance')
plt.xlabel('Attendance')
plt.ylabel('Average of Patients');

That means only 29% of patients that attende the appointment they received a reminding message, and the others just attende the appointment without a reminding message, and 43% of patients they received a message but didn't came for the appointment.
So, from the result above the reminding message does not affect on the attendance.

### Is the Difference between scheduled day and appointment day affect on Attendance ?

In [ ]:
df['Difference'] = df['AppointmentDay'].sub(df['ScheduledDay']).dt.days
df.head(10)

In [ ]:
df.groupby(['Difference'])['Attendance'].mean()

In [ ]:
df.groupby(['Difference'])['Attendance'].sum().plot(figsize=[16,5]);
plt.title('The Effect of Difference between Appointment day & Scheduled day on Attendance')
plt.xlabel('Difference')
plt.ylabel('Attende Patients');

That means a lot of patients who's scheduled the appointment at the same day of the appointment or near from the appointment day they attende. 

### The Age and Attendance

In [ ]:
attende = df.Attendance == True 
absente = df.Attendance == False

In [ ]:
def Attendance(df,thecolumn,Attende,Absente):
    plt.figure(figsize=[15,5]);
    df[thecolumn][attende].hist(color='blue',alpha=0.5,label=attende,bins=15)
    df[thecolumn][absente].hist(color='red',alpha=0.5,label=absente,bins=15)
    plt.legend();
    plt.title('The Age and Attendance')
    plt.xlabel('Age')
    plt.ylabel('Attende Patients');
Attendance(df,'Age',attende,absente)

from the figure above, the most attended patients their ages ranged from birth to 8.

# Limitation
The patients who attende the appointment more than non-attende
# Conclusions
The greater the difference between scheduled day and appointment day, the lower the attendance.

In [ ]:
from subprocess import call
call(['python', '-m', 'nbconvert', 'Investigate_a_Dataset.ipynb'])